In [1]:
import os
import tensorflow as tf
from tasks.tasks import SvAgreementLM, WordSvAgreementLM, WordSvAgreementVP
from tf2_models.lm_transformer import LmGPT2, ClassifierGPT2
from tf2_models.metrics import masked_sequence_loss
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.lm_lstm import LmLSTM, LmLSTMSharedEmb, ClassifierLSTM
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
from util import constants
from collections import Counter
from tqdm import tqdm
import numpy as np
from tf2_models.metrics import *

MODELS = {"lm_lstm": LmLSTM,
          "lm_gpt2": LmGPT2,
          "lm_lstm_shared_emb": LmLSTMSharedEmb,
          'cl_gpt2': ClassifierGPT2,
          'cl_lstm': ClassifierLSTM}

In [2]:
log_dir = "../logs"
chkpt_dir = "../tf_ckpts"
exp_name = "l2"

task = WordSvAgreementVP(task_params=get_task_params(),data_dir='../data')

Vocab len:  10034


In [3]:
model_config = 'small_gpt_v2'
model_name = 'cl_gpt2'
train_config ='radam_fast'
# Create the Model
model_params = get_model_params(task,model_name, model_config)
print("model_params: ", model_params.__dict__)

cl_token = task.databuilder.sentence_encoder().encode(constants.bos)
model = MODELS[model_name](hparams=get_model_params(task,model_name, model_config), cl_token=cl_token)

trainer_params = get_train_params(train_config)

log_dir = os.path.join(log_dir,task.name, model.model_name+"_"+str(model_config)+"_"+str(trainer_params.learning_rate)+"_"+exp_name)
ckpt_dir = os.path.join(chkpt_dir,task.name, model.model_name+"_"+str(model_config)+"_"+str(trainer_params.learning_rate)+"_"+exp_name)

trainer = Trainer(task=task,
                model=model,
                train_params=get_train_params('radam_fast'),
                log_dir=log_dir,
                ckpt_dir=ckpt_dir)

trainer.restore()

model config: small_gpt_v2
{'embedding_dim': 128, 'resid_pdrop': 0.1, 'embd_pdrop': 0.0, 'attn_pdrop': 0.1}
model_params:  {'vocab_size': 10036, 'n_ctx': 1024, 'n_positions': 1024, 'embedding_dim': 128, 'depth': 6, 'n_head': 8, 'resid_pdrop': 0.1, 'embd_pdrop': 0.0, 'attn_pdrop': 0.1, 'layer_norm_epsilon': 1e-05, 'initializer_range': 0.02, 'num_labels': 2, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'output_attentions': False, 'output_hidden_states': False}
model config: small_gpt_v2
{'embedding_dim': 128, 'resid_pdrop': 0.1, 'embd_pdrop': 0.0, 'attn_pdrop': 0.1}
Model: "classifier_gp_t2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (GPT2)           multiple                  2605568   
_________________________________________________________________
e2c (Dense)                  multiple  

In [4]:
trainer.model.evaluate(task.test_dataset, steps=100)

100/100 [==============================] - 3s 34ms/step - loss: 0.7542 - sequence_loss: 0.6738 - sparse_categorical_accuracy: 0.6100


[0.7542004257440567, 0.6737858, 0.61]

In [ ]:
for x,y in task.test_dataset:
    print(len(x))
    mask = tf.cast(y > 0, dtype=tf.float32)
    logits = model(x)
    print(logits.shape)
    logits = tf.reshape(logits, (-1, logits.shape[-1]))
    targets = tf.reshape(y, (-1, 1))
    mask = tf.reshape(mask, (-1, 1))
    correct = tf.cast(tf.argmax(model(x), axis=-1) == y, dtype=tf.float32)
    print(logits.shape)
    print(targets.shape)
    print(model.loss)
    print(model.loss(y_pred=logits, y_true=targets))
    print(masked_sequence_loss(y_pred=logits, y_true=targets))
    break

In [6]:
def gen_inflect_from_vocab(vocab_file, freq_threshold=1000):
    vbp = {}
    vbz = {}
    nn = {}
    nns = {}
    from_pos = {'NNS': nns, 'NN': nn, 'VBP': vbp, 'VBZ': vbz}

    for line in open(vocab_file):
        if line.startswith(' '):   # empty string token
            continue
        word, pos, count = line.strip().split()
        count = int(count)
        if len(word) > 1 and pos in from_pos and count >= freq_threshold:
            from_pos[pos][word] = count

    verb_infl = {'VBP': 'VBZ', 'VBZ': 'VBP'}
    for word, count in vbz.items():
        candidate = infl_eng.plural_verb(word)
        if candidate in vbp:
            verb_infl[candidate] = word
            verb_infl[word] = candidate

    noun_infl = {'NN': 'NNS', 'NNS': 'NN'}
    for word, count in nn.items():
        candidate = infl_eng.plural_noun(word)
        if candidate in nns:
            noun_infl[candidate] = word
            noun_infl[word] = candidate

    return verb_infl, noun_infl

In [7]:
from util import inflect

infl_eng = inflect.engine()

dependency_fields = ['sentence', 'orig_sentence', 'pos_sentence',
                     'subj', 'verb', 'subj_pos', 'has_rel', 'has_nsubj',
                     'verb_pos', 'subj_index', 'verb_index', 'n_intervening',
                     'last_intervening', 'n_diff_intervening', 'distance',
                     'max_depth', 'all_nouns', 'nouns_up_to_verb']

verb_infl, noun_infl = gen_inflect_from_vocab('wiki.vocab')

In [8]:
distance_hits = Counter()
distance_total = Counter()
diff_hits = Counter()
diff_total = Counter()

test_data = task.databuilder.as_dataset(split='test', batch_size=1000)
e = 0
for example in tqdm(test_data):
    e += 1
    verb_indexes = example['verb_position'] - 1
    encoded_sentences = example['sentence']
    mask = tf.cast(tf.sequence_mask(example['verb_position'],maxlen=tf.shape(encoded_sentences)[1]), dtype=tf.int64)
    encoded_sentences = encoded_sentences * mask
    max_length = tf.reduce_max(example['verb_position'])
    encoded_sentences = encoded_sentences[:,:max_length]
    s_shape = tf.shape(encoded_sentences)
    batch_size, length = s_shape[0], s_shape[1]
    verb_classes = example['verb_class']
    
    actual_verbs = example['verb']
    inflected_verbs = [verb_infl[v.decode("utf-8")] for v in actual_verbs.numpy()]
    
    distances = example['distance'].numpy()
    nz = example['n_intervening'].numpy()
    n_diffs = example['n_diff_intervening'].numpy()
    
    sentence =  task.databuilder.sentence_encoder().decode(encoded_sentences[0])
    actual_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in actual_verbs.numpy()]
    inflected_verb_indexes = [task.databuilder.sentence_encoder().encode(v)[0] for v in inflected_verbs]

    
    predictions = model(encoded_sentences)
    predictions = np.argmax(predictions, axis=-1)
    corrects = predictions == verb_classes

    for i, c in enumerate(corrects):
        if verb_indexes[i] == 10035:
            continue
        if nz[i] > 4 or distances[i] > 16:
            continue
            
        distance_total[distances[i]] += 1
        distance_hits[distances[i]] += int(c)
        if nz[i] == n_diffs[i]:
            n = nz[i]
            diff_total[n] += 1
            diff_hits[n] += int(c)
            

0it [00:00, ?it/s]

ResourceExhaustedError: OOM when allocating tensor with shape[1000,8,42,42] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Softmax]

In [ ]:
dis_acc = {}
dis_acc = np.zeros(17)
dif_acc = np.zeros(5)
print('Accuracy by distance')
for k in sorted(distance_hits.keys()):
    v = distance_hits[k]
    acc = v / distance_total[k]
    dis_acc[k] = acc
    print("%d | %.2f" % (k, acc), distance_total[k])

print('Accuracy by intervenings')
for k in sorted(diff_hits.keys()):
    v = diff_hits[k]
    acc = v * 1./diff_total[k]
    print("%d | %.2f" % (k, acc), diff_total[k])
    dif_acc[k] = acc

stats = {'distance': dis_acc, 'intervenings': dif_acc}

In [63]:
task.databuilder.sentence_encoder().encode("unk")

[10035]